# Test of VennABERS fast implementation

The notebook documents a test on the VennABERS implementation.

The correctness is tested against the slower implementation of Venn-ABERS predictors as per the definition.

In [1]:
import numpy as np

In [2]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
np.random.seed(0)

In [5]:
def sigmoid(x):
    return np.exp(-np.logaddexp(0,-x))

def thr(xs):
    return 0.5*(sigmoid((xs+4))+sigmoid(4*(xs-4)))

def classAssignment(xs):
    global thr
    u = np.random.random(size=xs.shape[0])
    ys = u<thr(xs)
    return ys

In [6]:
xs = np.linspace(-10,10,1000)
ys = classAssignment(xs)

In [7]:
p = figure(plot_width=600, plot_height=600)
p.line(xs,thr(xs),color='green')
p.scatter(xs,ys)
show(p)

In [8]:
from sklearn.isotonic import IsotonicRegression

import VennABERS

In [9]:
def VennABERS_by_def(ds,test):
    p0,p1 = [],[]
    for x in test:
        ds0 = ds+[(x,0)]
        iso0 = IsotonicRegression().fit(*zip(*ds0))
        p0.append(iso0.predict([x]))
        
        ds1 = ds+[(x,1)]
        iso1 = IsotonicRegression().fit(*zip(*ds1))
        p1.append(iso1.predict([x]))
    return np.array(p0).flatten(),np.array(p1).flatten()

In [10]:
xs = np.random.uniform(low=-10,high=10,size=400)
ys = classAssignment(xs)

In [11]:
xtest = np.linspace(-11,11,1000)

In [12]:
p0d,p1d = VennABERS_by_def(list(zip(xs,ys)),xtest)

In [13]:
p0,p1 = VennABERS.ScoresToMultiProbs(list(zip(xs,ys)),xtest)

In [14]:
discrepancies_p0 = np.argwhere(~np.isclose(p0.flatten(),p0d.flatten()))
discrepancies_p1 = np.argwhere(~np.isclose(p1.flatten(),p1d.flatten()))

print("p0: there are", discrepancies_p0.shape[0], "discrepancies")
print("p1: there are", discrepancies_p1.shape[0], "discrepancies")

p0: there are 0 discrepancies
p1: there are 0 discrepancies


In [15]:
np.any(p0>p1)

False

### Success! No discrepancy between fast implementation and reference implementation

Let's try with added duplicates (more than one point with the same score)

In [16]:
# Let's duplicate 1 in 10

x_dups = xs[::10]
y_dups = ~ys[::10]   # Let's switch labels, just for extra fun

In [17]:
xs = np.r_[xs,x_dups]
ys = np.r_[ys,y_dups]

In [18]:
xs.shape, ys.shape

((440,), (440,))

In [19]:
p0d,p1d = VennABERS_by_def(list(zip(xs,ys)),xtest)
p0f,p1f = VennABERS.ScoresToMultiProbs(list(zip(xs,ys)),xtest)

In [20]:
np.any(p0f>p1f)

False

In [21]:
discrepancies_p0f = np.argwhere(~np.isclose(p0f,p0d))
discrepancies_p1f = np.argwhere(~np.isclose(p1f,p1d))

print("p0f: there are", discrepancies_p0f.shape[0], "discrepancies")
print("p1f: there are", discrepancies_p1f.shape[0], "discrepancies")

p0f: there are 0 discrepancies
p1f: there are 0 discrepancies


### Success!! No discrepancy between reference implementation and fast implementation

Just out of curiosity, let's plot the calibrator.

In [22]:
xs = np.linspace(-11,11,220)

p = figure(plot_width=600, plot_height=600)
p.scatter(xtest,p0d,color='green')
p.scatter(xtest,p1d,color='red')
p.line(xs,thr(xs),color='blue')
show(p)

### If this does not look too good, consider that I introduced 10% of "adversarial" data, namely duplicates of points with their label flipped.